# Flowers-102 Image Classification

## Imports

In [152]:
import torch
import torchvision
#following import name conventions
import torchvision.transforms.v2 as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

## Processor choice

In [153]:
#Use GPU if possible
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

## Loading and Normalising Data

In [154]:
#Transform to be ran on data
transforms = transforms.Compose([
    transforms.RandomCrop(size=(244,244)),
    transforms.RandomHorizontalFlip(), #reduce bias with image flip
    transforms.RandomRotation(10), #reduce bias with random angles
    
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # placeholder until mean and std of dataset calculated
])

c:\Users\nicho\anaconda3\lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [155]:
#Hyperparameters
batch_size = 64
learning_rate = 0.001
epoch_count = 10

In [156]:
train_data = torchvision.datasets.Flowers102(
    root="./data",
    download=True,
    split= "train",
    transform=transforms
)
test_data = torchvision.datasets.Flowers102(
    root="./data",
    download=True,
    split= "test",
    transform=transforms
)

In [157]:
train_loader = torch.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=batch_size)

## Convolutional Neural Network

In [158]:
class cnn(nn.Module):
    def __init__(self):
        #Layers
        
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # params are input channels, output channels, filter(kernel) size
        self.pool = nn.MaxPool2d(2,2) # params are kernal size, stride
        self.conv2 = nn.Conv2d(6, 16, 5)

        self.lin1 = nn.Linear(53824,120) # params are input features, output features
        self.lin2 = nn.Linear(120, 84)
        self.lin3 = nn.Linear(84,102) #output features must be equal to num of categories
    
    def forward(self,x):
        """Called to pass input data through layers, uses activation function"""
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = torch.flatten(x, 1)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)
        return x

model = cnn()



## Loss function and optimizer

In [159]:
loss_fn = nn.CrossEntropyLoss() #  probibalistic good for multiclass uses softmax
optimizer = optim.SGD(model.parameters(),learning_rate) #Stochastic gradient descent

## Training loop

In [164]:
for epoch in range(epoch_count): #loop multiple times
    running_loss = 0.0
    for i, (images,labels) in enumerate(train_loader): # pulls out inputs and labels from training data
        optimizer.zero_grad() #zero parameter gradients
        
        #forward cnn, backpropagation (backward and optimize)
        outputs = model(images)
        loss = loss_fn(outputs,labels)
        loss.backward()
        optimizer.step()
        
        #print statistics
        running_loss += loss.item()
    print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}")    #work out 2000 bit   
print("Training complete")

[1,    16] loss: 0.037


[2,    16] loss: 0.037
[3,    16] loss: 0.037


KeyboardInterrupt: 

## Test model

In [162]:
correct = 0
total = 0
accuracy = 0

with torch.no_grad(): # in testing, don't need to calculate gradients for ouputs
    for (images,labels) in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data,1) # returns predicted class labels in one dimension
        total += labels.size(0) #adds number of samples in batch to total sample count
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct // total
print(f"Accuracy of the network on the test images: {test_accuracy} %")

Accuracy of the network on the test images: 1 %


## Save model

In [163]:
torch.save(model.state_dict(), f"models/model_accuracy_{test_accuracy}.pt")